In [ ]:
# install pytorch geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.9.0+cu111.html --quiet

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from google.colab import drive
drive.mount('/content/drive')
device='cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.get_device_name(0))

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tesla K80


In [ ]:
# pseudo graph
# with open('/content/drive/MyDrive/gnn/geometric/118_edge_index.txt','w') as f:
#   for source in range(118):
#     for target in range(118):
#       if source!=target:
#         f.write('%d\t%d\n'%(source,target))
#         f.write('%d\t%d\n'%(target,source))

In [ ]:
# read edge index
total=open('/content/drive/MyDrive/gnn/geometric/118_edge_index.txt','r').readlines()
edges=[[],[]]
for line in total:
  [source,target]=line.strip().split()
  edges[0].append(int(source))
  edges[1].append(int(target))
edge_index=torch.tensor(edges,dtype=torch.long)
x=np.load('/content/drive/MyDrive/gnn/data/dc118_p10_x_congest1.npy').transpose(2,0,1)
y=np.load('/content/drive/MyDrive/gnn/data/dc118_p10_y_congest1.npy').transpose()
y=np.expand_dims(y,axis=2)
W=np.load('/content/drive/MyDrive/gnn/data/dc118_p10_w.npy')

In [ ]:
# x and y are combined in PyG data
data=[]
for idx in range(y.shape[0]):
  graph=Data(x=torch.tensor(x[idx],dtype=torch.float),y=torch.tensor(y[idx],dtype=torch.float))
  data.append(graph)
train=data[:int(len(data)*0.8)]
test=data[int(len(data)*0.8):]
train_loader=DataLoader(train,batch_size=32,shuffle=True)
test_loader=DataLoader(test,batch_size=32,shuffle=True)

In [ ]:
class GCN(torch.nn.Module):
  def __init__(self,edge_index,hidden,num_bus):
    super().__init__()
    self.edge_index=edge_index
    self.conv=[GCNConv(-1,hidden[0])]
    for idx in range(len(hidden)-1):
      self.conv.append(GCNConv(hidden[idx],hidden[idx+1]))
    self.linear=nn.Linear(hidden[-1],num_bus)
  
  def forward(self,input):
    x=input.x
    for layer in self.conv:
      x=layer.forward(x,self.edge_index)
    return x
num_bus=y.shape[-1]
hidden=[5,10,1]
net=GCN(edge_index,hidden,num_bus)
net=net

loss_func=nn.MSELoss()
optimizer=torch.optim.Adam(net.parameters(),weight_decay=0.01)

In [ ]:
max_epochs=100
eval_epoch=10

# earlystopping
tolerance= 4
min_delta=5e-4
previous=0

loss_train=[]
loss_eval=[]
for epoch in range(max_epochs):
  # training loop
  train_loss=0.0
  for batch in train_loader:
    batch=batch
    optimizer.zero_grad()
    logits=net(batch)
    loss=loss_func(logits,batch.y)
    loss.backward()
    train_loss+=loss.item()
    optimizer.step() # update parameters of net
  train_avg=train_loss/len(train_loader.dataset)
  loss_train.append(train_avg)
  print("Epoch %d | Training loss: %.4f"%(epoch,train_avg))
  # eval
  if (epoch+1)%eval_epoch==0:
    net.eval()
    eval_loss=0.0
    for batch in test_loader:
      batch=batch
      logits=net(batch)
      loss=loss_func(logits,batch.y)
      eval_loss+=loss.item()
    eval_avg=eval_loss/len(test_loader.dataset)
    if (epoch==0): previous=eval_avg
    else:
      if previous-eval_avg<min_delta: tolerance-=1
      if tolerance==0: break
      previous=eval_avg
    print("Epoch %d | Eval loss: %.4f" % (epoch,eval_avg))
    loss_eval.append([epoch,eval_avg])
    net.train()

Epoch 0 | Training loss: 24.8821
Epoch 1 | Training loss: 24.8828
Epoch 2 | Training loss: 24.8822
Epoch 3 | Training loss: 24.8865
Epoch 4 | Training loss: 24.8796
Epoch 5 | Training loss: 24.8807
Epoch 6 | Training loss: 24.8836
Epoch 7 | Training loss: 24.8849
Epoch 8 | Training loss: 24.8822
Epoch 9 | Training loss: 24.8867
Epoch 9 | Eval loss: 26.3503
Epoch 10 | Training loss: 24.8839


KeyboardInterrupt: ignored

In [ ]:

import matplotlib.pyplot as plt

val_len = len(loss_eval)
print(val_len)
val_plt = np.zeros((2,val_len))
for i in range(val_len):
  val_plt[0,i] = loss_eval[i][0]
  val_plt[1,i] = loss_eval[i][1]

plt.figure()
plot_idx = np.arange(np.size(loss_optm))
# plt.plot(plot_idx,loss_optm,lw=2,label='training loss')
# plt.plot(val_plt[0,:],val_plt[1,:],lw=2,label='validation loss')
plt.plot(plot_idx[5:-1],loss_optm[5:-1],lw=2,label='training loss')
plt.plot(val_plt[0,1:],val_plt[1,1:],lw=2,label='validation loss')
plt.yscale("log")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show(block=False)

In [ ]:
n_test = 2000
x_test_feed = torch.from_numpy(np.transpose(x_test)).float()
x_test_feed = x_test_feed.to(device)
y_pred = net(x_test_feed)
print('Validation dataset size:',x_test_feed.shape)
print('Number of validation set: ',n_test)
y_pred1=y_pred.cpu().detach()
y_pred1=torch.squeeze(y_pred1,1).numpy().transpose()
print(y_test.shape)

n_test = np.size(y_test,1)
err_L2 = np.zeros(n_test)
err_Linf = np.zeros(n_test)
for i in range(n_test):
  err_L2[i] = np.linalg.norm(y_test[:,i] - y_pred1[:,i]) / np.linalg.norm(y_test[:,i])
  err_Linf[i] = np.max(np.abs(y_test[:,i] - y_pred1[:,i])) / np.max(np.abs(y_test[:,i]))

In [ ]:
err_L2_mean = np.mean(err_L2)
err_Linf_mean = np.mean(err_Linf)
print('L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )

fig2 = plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
# plt.hist(np.abs(ga),bins = 10)
plt.plot(err_L2,'bo',markersize=0.5,label = 'L2 error')
plt.plot(err_Linf,'r^',markersize=0.5,label = 'Linf error')
plt.legend(loc="upper right")
plt.xlabel('sample index')
plt.ylabel('error')
plt.title('Normalized sample error')
plt.grid(True)
# error histogram
plt.subplot(1, 2, 2)
plt.hist(err_L2, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
plt.hist(err_Linf, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
plt.legend(loc="upper right")
plt.xlabel('sample error value')
plt.ylabel('frequency')
plt.title('Histogram of L_2 error')
# plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.grid(True)
plt.show()

In [ ]:
index = 1
y_test_copy = y_pred1[:,index].copy()
print(y_test_copy.shape)
for i in range(118):
  if y_test_copy[i] < 0.1:
    y_test_copy[i] = y_test[i,index] + 50
print(np.linalg.norm(y_test_copy - y_test[:,index]) / np.linalg.norm(y_test[:,index]))

# Predict generation using $\pi$
* Using predicted $\pi$ and find the active constraints in $p_G(i)$
* For inactive $p_G(i)$ consider other methods like power flow balance

In [ ]:
gen_limit0 = x[:,2,:].copy()
print(gen_limit0.shape)

gen_idx = []
for i in range(n_bus):
  if gen_limit0[i,0] > 0:
    gen_idx.append(i)
print(type(gen_idx),len(gen_idx),gen_idx)

In [ ]:
# # get the generator index
# # generator cost data
# gen_cost0
# # generator limit data
# gen_limit0
# # LMP data
# lmp_data
# # generation data
# gen_data

load_data = x.copy()
print(load_data.shape)
n_sample = np.size(load_data,2)

x_val_feed = torch.from_numpy(np.transpose(load_data)).float()
x_val_feed = x_val_feed.to(device)

print('Dataset size:',x_val_feed.shape)
print('Number of validation points:: ',n_sample)
y_pred = net(x_val_feed) # predict the corredponding LMP

y_pred1 = y_pred.cpu().detach()
y_pred1 = torch.squeeze(y_pred1,1).numpy().transpose()

* Save results

In [ ]:
# dataset = {'lmp_pred': y_pred1, 'lmp_true': lmp_data}
# print('Pred:',y_pred1.shape,'True:',lmp_data.shape)

# file_name = 'dc118_lmp_prediction_p10'
# file_path = 'drive/My Drive/gnn/data/results/'
# file_dir = file_path + file_name + '.pickle'
# outfile = open(file_dir, 'wb')
# pickle.dump(dataset, outfile)
# outfile.close()

In [ ]:
import pickle
gen_cost0 = x[:,1,:].copy()
lmp_data = y.copy()

profit_pred = y_pred1 - gen_cost0
print(np.min(np.abs(profit_pred)))

profit_true = lmp_data - gen_cost0

gen_pred_binary = np.zeros((len(gen_idx),n_sample))
gen_true_binary = np.zeros((len(gen_idx),n_sample))
print(gen_pred_binary.shape)

binary_thres = 0.97
binary_thres_true = 1e-5

for i in range(n_sample):
  for j in range(len(gen_idx)):
    # predicted generator limit
    if profit_pred[gen_idx[j],i] > binary_thres:
      gen_pred_binary[j,i] = 1
    elif profit_pred[gen_idx[j],i] < 1-binary_thres:
      gen_pred_binary[j,i] = 0
    else:
      gen_pred_binary[j,i] = 0.5
    # true generator limit
    if profit_true[gen_idx[j],i] > binary_thres_true:
      gen_true_binary[j,i] = 1
    elif profit_true[gen_idx[j],i] < 1e-5:
      gen_true_binary[j,i] = 0
    else:
      gen_true_binary[j,i] = 0.5


In [ ]:
gen_binary_err = np.abs(gen_true_binary - gen_pred_binary)
print('max binary error:',np.max(gen_binary_err))
# count the wrong entries
gen_binary_err_ct = np.sum(gen_binary_err)
gen_binary_err_ratio = gen_binary_err_ct / (len(gen_idx)*n_sample)
print('Binary accuracy:',1-gen_binary_err_ratio)

In [ ]:
print(gen_limit0.shape)
print(np.transpose(x_test).shape,np.transpose(load_data).shape)